In [46]:
#Import libraries
import sklearn 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# import tensorflow as tf  # acts as the framework upon which this model is built
import numpy as np  # basic scientific computation
import pandas as pd #for data science
import scipy
import matplotlib.pyplot as plt #ploting 
import seaborn as sns; sns.set_style('darkgrid') #ploting
my_dpi=96 #dots per inch. Used in the size of the plots
 
import csv #library for handling csv files 




In [58]:
df = pd.read_csv('bank/bank-full.csv', sep=';')

In [60]:

# Group age by percentages of 10
labels = [0] + [f'{i+10}' for i in range(10, 100, 10)]
df['age_groups'] = pd.qcut(df['age'], 10, labels=labels)
df.age_groups = df.age_groups.astype("int")
# print(df[['age', 'age_groups']])


In [61]:
# Split useable and target group
useColumns = df[['age', 'age_groups', 'job', 'marital',  'education', 'default', 'balance', 'housing', 'loan',
                 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']]
target = df[['y']]


In [64]:

transformer = make_column_transformer(
    (OneHotEncoder(), ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']),
    remainder='passthrough')
transformed = transformer.fit_transform(useColumns)
transformed_df = pd.DataFrame(
    transformed, columns=transformer.get_feature_names())
# print(transformed_df.head())


c:\ProgramData\Anaconda3\envs\thisone\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [56]:
# Correlation of variables.
# df.corr()

In [66]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(transformed_df, target)


In [67]:
clf

DecisionTreeClassifier()

In [55]:
# enc = OneHotEncoder(handle_unknown='ignore')

# job = [["admin.",1], ["unknown",2], ["unemployed",3], ["management",4], ["housemaid",5], ["entrepreneur",6], ["student",7], ["blue-collar",8], ["self-employed",9], ["retired",10], ["technician",11], ["services",12]]
# marital = [["married",1], ["divorced",2], ["single",3]]
# education = [["unknown",1], ["secondary",2], ["primary",3], ["tertiary",4]]
# default = [["yes",1], ["no",2]]
# housing = [["yes",1], ["no",2]]
# loan = [["yes", 1], ["no", 2]]
# contact = [["unknown",1], ["telephone",2], ["cellular",3]]
# month = [["jan",1], ["feb",2], ["mar",3], ["apr",4], ["may",5], ["jun",6], ["jul",7], ["aug",8], ["sep",9], ["oct",10], ["nov",11], ["dec",12]]
# poutcome = [["unknown", 1], ["other", 2], ["failure", 3], ["success", 4]]
# y =[["yes", 1], ["no", 2]]


# # enc.fit([job, marital, education, default, housing, loan, contact, month, poutcome, y])
# enc.fit(job)

# enc.categories_
# enc.transform(job).toarray()

# enc.inverse_transform([[0, 1, 1, 0, 0, 3, 4, 6, 3, 2, 7, 12], [
#                       0, 0, 0, 1, 0, 11, 10, 9, 9,8,7]])

# # enc.get_feature_names_out(['gender', 'group'])


In [8]:
# pipe = make_pipeline(StandardScaler(), LogisticRegression(), verbose=True)

# X, y = load_iris(return_X_y=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# pipe.fit(X_train, y_train)
# accuracy_score(pipe.predict(X_test), y_test)


In [9]:
# ageBins = [18, 34, 50, 70, 88, 95]
# df['age_groups'] = pd.cut(df['age'], ageBins)
# print(df[['age', 'age_groups']])


In [57]:
# Switch all categorical vallues to numerical
# useColumns.job = useColumns.job.replace(["admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student",
#                         "blue-collar", "self-employed", "retired", "technician", "services"], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
# useColumns = pd.get_dummies(useColumns, columns=['job'], prefix='job', prefix_sep='')


# useColumns.marital = useColumns.marital.replace(["married", "divorced", "single"], [1, 2, 3])
# useColumns = pd.get_dummies(useColumns, columns=['marital'], prefix='marital', prefix_sep='')

# df.education = df.education.replace(
#     ["unknown", "secondary", "primary", "tertiary"], [1, 2, 3, 4])
# df = pd.get_dummies(df, columns=['education'],
#                     prefix='education', prefix_sep='')

# df.default = df.default.replace(["yes", "no"], [1, 2])
# df = pd.get_dummies(df, columns=['default'], prefix='default', prefix_sep='')

# df.housing = df.housing.replace(["yes", "no"], [1, 2])
# df = pd.get_dummies(df, columns=['housing'], prefix='housing', prefix_sep='')

# df.loan = df.loan.replace(["yes", "no"], [1, 2])
# df = pd.get_dummies(df, columns=['loan'], prefix='loan', prefix_sep='')

# df.contact = df.contact.replace(
#     ["unknown", "telephone", "cellular"], [1, 2, 3])
# df = pd.get_dummies(df, columns=['contact'], prefix='contact', prefix_sep='')

# df.month = df.month.replace(["jan", "feb", "mar", "apr", "may", "jun", "jul",
#                             "aug", "sep", "oct", "nov", "dec"], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
# # df = pd.get_dummies(df, columns=['month'], prefix='month', prefix_sep='')

# df.poutcome = df.poutcome.replace(
#     ["unknown", "other", "failure", "success"], [2, 3, 0, 1])
# df = pd.get_dummies(df, columns=['poutcome'], prefix='poutcome', prefix_sep='')

# df.y = df.y.replace(["yes", "no"], [1, 0])
# df = pd.get_dummies(df, columns=['y'], prefix='y', prefix_sep='')


# # print(df.isna().sum())
